In [ ]:
# You may need to install pyamg
!pip3 install pyamg

In [ ]:
import numpy as np
import scipy.io as sio
import pyamg
import scipy.sparse.linalg as sla

import matplotlib.pyplot as plt
from matplotlib import collections
from matplotlib import tri
%matplotlib inline

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Anisotropic diffusion problem

In [ ]:
stencil = pyamg.gallery.diffusion.diffusion_stencil_2d(type='FE', epsilon=0.001, theta=2 * np.pi / 16.0)
A = pyamg.gallery.stencil_grid(stencil, (50, 50), format='csr')

# Candidate vectors

Start with a single vector

In [ ]:
B = np.ones((A.shape[0], 1))

In [ ]:
ml = pyamg.smoothed_aggregation_solver(A, B, max_coarse=1)
print(ml)

In [ ]:
res = []
x = ml.solve(np.random.rand(A.shape[0]), residuals=res)
res = np.array(res)
res[1:] / res[:-1]

Convergence is poor, what does the solution (error) look like?

In [ ]:
len(res)

In [ ]:
plt.pcolormesh(x.reshape(50,50))
plt.colorbar()

Can we augment the candidate vector?  Take a vector of ones and the old solution (error) vector.

In [ ]:
B = np.vstack((np.ones(A.shape[0],), x)).T

In [ ]:
B.shape

In [ ]:
ml = pyamg.smoothed_aggregation_solver(A, B, max_coarse=1, keep=True)

In [ ]:
print(ml)

In [ ]:
res = []
x = ml.solve(np.random.rand(A.shape[0]), residuals=res)
res = np.array(res)
res[1:] / res[:-1]

In [ ]:
plt.pcolormesh(x.reshape(50,50))

In [ ]:
ml

In [ ]:
ml.levels[0].P.shape

In [ ]:
ml.levels[0].AggOp.shape

# Try Adaptive AMG

In short, adaptive AMG will run a cycle on $Ax=0$ to determine improved $B$ vectors

In [ ]:
mladapt, work = pyamg.aggregation.adaptive_sa_solver(A, num_candidates=3)
print(mladapt)

Let's look at the candidate vectors

In [ ]:
plt.pcolormesh(mladapt.levels[0].B[:,0].reshape(50,50))

In [ ]:
plt.pcolormesh(mladapt.levels[0].B[:,1].reshape(50,50))

In [ ]:
plt.pcolormesh(mladapt.levels[0].B[:,2].reshape(50,50))

In [ ]:
ml = pyamg.smoothed_aggregation_solver(A, mladapt.levels[0].B, max_coarse=1)
print(ml)

# Check the convergence

Let's see how well improved candidate vectors work for both a SA and adaptive SA method.

In [ ]:
res = []
x = ml.solve(np.random.rand(A.shape[0]), residuals=res)
res = np.array(res)
res[1:] / res[:-1]

In [ ]:
res = []
x = mladapt.solve(np.random.rand(A.shape[0]), residuals=res)
res = np.array(res)
res[1:] / res[:-1]

# Improving SOC

In [ ]:
B = np.ones((A.shape[0],1))
ml = pyamg.smoothed_aggregation_solver(A, B, strength='evolution', max_coarse=5)
print(ml)

In [ ]:
res = []
x = ml.solve(np.random.rand(A.shape[0]), residuals=res)
res = np.array(res)
res[1:] / res[:-1]

# Improving Interpolation

In [ ]:
B = np.ones((A.shape[0],1))
smooth=('energy', {'krylov': 'cg', 'maxiter': 4, 'degree': 3, 'weighting': 'local'})
ml = pyamg.smoothed_aggregation_solver(A, B, strength='evolution', max_coarse=5,
                                       smooth=smooth)
print(ml)

In [ ]:
res = []
x = ml.solve(np.random.rand(A.shape[0]), residuals=res)
res = np.array(res)
res[1:] / res[:-1]

In [ ]:
res